In [122]:
import pandas as pd
import numpy as np

In [123]:
df = pd.read_csv("./UH_2023/UH_2023_TRAIN.txt", sep = "|")

In [124]:
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str})

In [125]:
def fun(x, mode): 
    altitud = str(x).split('-')
    if altitud[0] == "nan":
        if mode == "nan_1":
            return -1
        return np.nan
    if mode == "nan_1":
            return x
    if mode == "min":
        return int(altitud[0])
    if mode == "diff":
        if len(altitud) == 1:
            return 0
        else:
            return int(altitud[1])-int(altitud[0])

df["ALTITUD_MIN"] = df["ALTITUD"].apply(fun, args=("min",))
df["ALTITUD_DIF"] = df["ALTITUD"].apply(fun, args=("diff",))
df["ALTITUD"] = df["ALTITUD"].apply(fun, args=("nan_1",))

In [126]:
df_estacion_gb_mean = df.groupby(by="ID_ESTACION").mean()
df_estacion_gb_mode = df.groupby(by="ID_ESTACION").agg(pd.Series.mode)

C:\Users\xavid\anaconda3\lib\site-packages\pandas\core\algorithms.py:968: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  warn(f"Unable to sort modes: {err}")
C:\Users\xavid\AppData\Local\Temp\ipykernel_8044\3219566741.py:2: FutureWarning: ['SUPERFICIE', 'PRODUCCION', 'ALTITUD_DIF'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df_estacion_gb_mode = df.groupby(by="ID_ESTACION").agg(pd.Series.mode)


In [127]:
df_estacion_gb_mean.reset_index(inplace=True)
df_estacion_gb_mode.reset_index(inplace=True)

In [128]:
df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == "0", "ALTITUD"] = 390
df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == "4", "ALTITUD"] = 520

In [129]:
for i in df["ID_ESTACION"].unique():
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD_MIN"] = df_estacion_gb_mean.loc[df_estacion_gb_mean["ID_ESTACION"] == i,"ALTITUD_MIN"].values[0]
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD_DIF"] = df_estacion_gb_mean.loc[df_estacion_gb_mean["ID_ESTACION"] == i,"ALTITUD_DIF"].values[0]
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD"] = df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == i,"ALTITUD"].values[0]

In [130]:
df

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
0,14,76953,515,4,660,26,2,0,1,0.0000,22215.0,660.000000,NaN
1,14,84318,515,4,660,26,2,0,1,0.0000,22215.0,660.000000,NaN
2,14,85579,340,4,520,32,2,0,1,0.0000,20978.0,520.000000,NaN
3,14,69671,340,4,520,32,2,0,1,0.0000,40722.0,520.000000,NaN
4,14,14001,852,14,650-660,81,1,0,1,0.0000,14126.0,654.501718,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9596,22,37461,239,6,700,52,2,0,1,3.6800,NaN,700.000000,NaN
9597,22,58769,239,6,700,32,2,0,1,4.2500,NaN,700.000000,NaN
9598,22,58769,239,6,700,59,2,0,1,4.0700,NaN,700.000000,NaN
9599,22,88928,239,6,700,40,2,0,1,4.5727,NaN,700.000000,NaN


In [10]:
import category_encoders as ce

encoder = ce.OneHotEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD"])

df_encoded = encoder.fit_transform(df)

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [11]:
df_encoded.head()

,CAMPAÑA,ID_FINCA_1,ID_FINCA_2,ID_FINCA_3,ID_FINCA_4,ID_FINCA_5,ID_FINCA_6,ID_FINCA_7,ID_FINCA_8,ID_FINCA_9,...,VARIEDAD_23,VARIEDAD_24,VARIEDAD_25,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
0,14,1,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,0.0,22215.0,660.000000,0.00000
1,14,0,1,0,0,0,0,0,0,0,...,0,0,0,2,0,1,0.0,22215.0,660.000000,0.00000
2,14,0,0,1,0,0,0,0,0,0,...,0,0,0,2,0,1,0.0,20978.0,520.000000,0.00000
3,14,0,0,0,1,0,0,0,0,0,...,0,0,0,2,0,1,0.0,40722.0,520.000000,0.00000
4,14,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0.0,14126.0,654.501718,9.19244


In [45]:
data = df_encoded.drop(index=df_encoded[df_encoded["CAMPAÑA"] == "22"].index)
without_zeros = df[(df["PRODUCCION"] != 0) & (df["SUPERFICIE"] != 0)]
with_zeros = df[(df["PRODUCCION"] == 0) | (df["SUPERFICIE"] == 0)]

In [64]:
uniquees = without_zeros["ID_FINCA"].unique()

In [67]:
a = df[~df["ID_FINCA"].isin(uniquees)]

In [76]:
df[df["ID_FINCA"] == "86582"] 

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
46,14,86582,464,14,650-660,17,1,0,1,0.0,90640.0,650.0,10.0
47,14,86582,464,14,650-660,59,1,0,1,0.0,14324.2,650.0,10.0
1190,15,86582,464,14,650-660,17,1,0,1,0.0,63540.0,650.0,10.0
1191,15,86582,464,14,650-660,59,1,0,1,0.0,14473.8,650.0,10.0
2306,16,86582,464,14,650-660,17,1,0,1,0.0,110120.0,650.0,10.0
2307,16,86582,464,14,650-660,59,1,0,1,0.0,7384.8,650.0,10.0
3386,17,86582,464,14,650-660,17,1,0,1,0.0,29095.0,650.0,10.0
3387,17,86582,464,14,650-660,59,1,0,1,0.0,4409.8,650.0,10.0
3388,17,86582,464,14,650-660,17,2,0,1,0.0,951.5,650.0,10.0
4404,18,86582,464,14,650-660,17,1,0,1,0.0,34413.5,650.0,10.0


In [73]:
b= a.groupby(by=["CAMPAÑA", "ID_FINCA"]).()
b

ID_ZONA  ID_ESTACION  ALTITUD  VARIEDAD  MODO  TIPO  COLOR  \
CAMPAÑA ID_FINCA                                                               
14      10311           1            1        1         1     1     1      1   
        10316           1            1        1         1     1     1      1   
        10602           1            1        1         1     1     1      1   
        10718           1            1        1         1     1     1      1   
        10838           1            1        1         1     1     1      1   
...                   ...          ...      ...       ...   ...   ...    ...   
20      86582           2            2        2         2     2     2      2   
        89426           1            1        1         1     1     1      1   
21      48093           1            1        1         1     1     1      1   
        86582           2            2        2         2     2     2      2   
22      86582           2            2        2         2     2     2      2   

                  SUPERFICIE  PRODUCCION  ALTITUD_MIN  ALTITUD_DIF  
CAMPAÑA ID_FINCA                                                    
14      10311              1           1            1            1  
        10316              1           1            1            1  
        10602              1           1            1            1  
        10718              1           1            1            1  
        10838              1           1            1            1  
...                      ...         ...          ...          ...  
20      86582              2           2            2            2  
        89426              1           1            1            1  
21      48093              1           1            1            1  
        86582              2           2            2            2  
22      86582              2           0            2            2  

[873 rows x 11 columns]

In [22]:
X = to_train.drop(axis = 1, columns = ["SUPERFICIE"])
y = to_train["SUPERFICIE"]
to_predict = to_predict.drop(axis = 1, columns = ["SUPERFICIE"])

In [29]:
X

,CAMPAÑA,ID_FINCA_1,ID_FINCA_2,ID_FINCA_3,ID_FINCA_4,ID_FINCA_5,ID_FINCA_6,ID_FINCA_7,ID_FINCA_8,ID_FINCA_9,...,VARIEDAD_22,VARIEDAD_23,VARIEDAD_24,VARIEDAD_25,MODO,TIPO,COLOR,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
6476,20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,1,19511.107,600.0,0.0
6477,20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,1,38678.893,600.0,0.0
6478,20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,7092.800,650.0,10.0
6480,20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,1,6660.000,650.0,10.0
6481,20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,1,6970.000,650.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8521,21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,1,28160.100,700.0,0.0
8522,21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,1,41310.000,700.0,0.0
8523,21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,1,45420.000,700.0,0.0
8524,21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,1,56140.000,700.0,0.0


In [24]:
y

6476    3.7503
6477    7.3735
6478    7.6200
6480    7.1657
6481    3.5000
         ...  
8521    3.6800
8522    4.2500
8523    4.1600
8524    4.7500
8525    1.4620
Name: SUPERFICIE, Length: 2017, dtype: float64

In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor


model = {"model_name": "SVC", "model": MLPRegressor(), "params": {'hidden_layer_sizes':[(100),(100,100,100)],
                                                                  'learning_rate':["constant", "adaptive"],}}

grid_search = GridSearchCV(model["model"], model["params"], scoring="r2")
grid_search.fit(X, y)

GridSearchCV(estimator=MLPRegressor(),
             param_grid={'hidden_layer_sizes': [100, (100, 100, 100)],
                         'learning_rate': ['constant', 'adaptive']},
             scoring='r2')

In [28]:
print(grid_search.best_params_)
df = pd.DataFrame(grid_search.cv_results_)
df

{'hidden_layer_sizes': 100, 'learning_rate': 'adaptive'}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,9.224663,1.973761,0.099098,0.008882,100,constant,"{'hidden_layer_sizes': 100, 'learning_rate': '...",0.445728,-0.420965,0.453176,0.437508,0.628864,0.308862,0.371795,2
1,8.290641,1.837650,0.102070,0.005233,100,adaptive,"{'hidden_layer_sizes': 100, 'learning_rate': '...",0.370585,0.474343,0.489454,0.606086,0.495627,0.487219,0.074799,1
2,6.724814,0.647214,0.104612,0.007782,"(100, 100, 100)",constant,"{'hidden_layer_sizes': (100, 100, 100), 'learn...",0.461523,0.346288,-0.014162,-0.017838,0.578257,0.270813,0.245405,3
3,8.539767,3.255845,0.100729,0.005617,"(100, 100, 100)",adaptive,"{'hidden_layer_sizes': (100, 100, 100), 'learn...",0.479489,-12.949451,0.493603,0.440723,0.592353,-2.188656,5.380629,4


In [16]:
modelMLPR = MLPRegressor(hidden_layer_sizes=(100), learning_rate="adaptive")
modelMLPR.fit(X, y)
predicted = modelMLPR.predict(to_predict)

In [29]:
final_df = to_predict
final_df["SUPERFICIE"] = predicted

In [30]:
final_df

,CAMPAÑA,ID_FINCA_1,ID_FINCA_2,ID_FINCA_3,ID_FINCA_4,ID_FINCA_5,ID_FINCA_6,ID_FINCA_7,ID_FINCA_8,ID_FINCA_9,...,VARIEDAD_23,VARIEDAD_24,VARIEDAD_25,MODO,TIPO,COLOR,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF,SUPERFICIE
0,14,1,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,22215.0,660.000000,0.00000,3.696552
1,14,0,1,0,0,0,0,0,0,0,...,0,0,0,2,0,1,22215.0,660.000000,0.00000,3.696552
2,14,0,0,1,0,0,0,0,0,0,...,0,0,0,2,0,1,20978.0,520.000000,0.00000,3.241230
3,14,0,0,0,1,0,0,0,0,0,...,0,0,0,2,0,1,40722.0,520.000000,0.00000,5.634995
4,14,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,14126.0,654.501718,9.19244,2.846990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6471,19,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,22947.5,700.000000,0.00000,3.769978
6472,19,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,30840.0,700.000000,0.00000,4.595498
6473,19,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,24090.0,700.000000,0.00000,3.879738
6474,19,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,35920.0,700.000000,0.00000,5.211837


In [32]:
final_df = pd.concat([final_df, to_train])

In [33]:
final_df

,CAMPAÑA,ID_FINCA_1,ID_FINCA_2,ID_FINCA_3,ID_FINCA_4,ID_FINCA_5,ID_FINCA_6,ID_FINCA_7,ID_FINCA_8,ID_FINCA_9,...,VARIEDAD_23,VARIEDAD_24,VARIEDAD_25,MODO,TIPO,COLOR,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF,SUPERFICIE
0,14,1,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,22215.0,660.000000,0.00000,3.696552
1,14,0,1,0,0,0,0,0,0,0,...,0,0,0,2,0,1,22215.0,660.000000,0.00000,3.696552
2,14,0,0,1,0,0,0,0,0,0,...,0,0,0,2,0,1,20978.0,520.000000,0.00000,3.241230
3,14,0,0,0,1,0,0,0,0,0,...,0,0,0,2,0,1,40722.0,520.000000,0.00000,5.634995
4,14,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,14126.0,654.501718,9.19244,2.846990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8521,21,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,28160.1,700.000000,0.00000,3.680000
8522,21,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,41310.0,700.000000,0.00000,4.250000
8523,21,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,45420.0,700.000000,0.00000,4.160000
8524,21,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,56140.0,700.000000,0.00000,4.750000
